# Telemática

En este cuaderno se presenta una guía de configuración de algunos servicios en máquinas Ubuntu en la plataforma AWS.

Realizado por:

Ferney O. Amaya Fernández <BR>
ferney.amaya@gmail.com

#1.Regiones, Zonas de Disponibilidad y VPCs

Amazon cuente con regiones en todo el mundo, por ejemplo:
- us-east-1 (Norte de Virginia, EE. UU.) con 6 zonas de disponibilidad
- eu-west-1 (Irlanda)
- ap-southeast-1 (Singapur)

En cada región se tienen varias Zonas de Disponibilidad (*Availability Zones, AZ*). Una AZ es un grupo de centros de datos con alimentación, refrigeración y seguridad física independientes.

[Mayor información aquí.](https://aws.amazon.com/es/about-aws/global-infrastructure/)

En una VPC (*Virtual Private Cloud*) pueden lanzarse varios recursos de AWS en una red virtual lógica aislada que se asemeja a una red tradicional que conforma un centro de datos.

[Mayor información aquí.](https://docs.aws.amazon.com/es_es/vpc/latest/userguide/what-is-amazon-vpc.html)

#2 Paso a paso para crear una VPC manual

La VPC y sus componentes pueden crearse en forma automática y en forma manual. Aquí se presenta una breve guía para crearlas en forma manual.

Se creará una VPC con 1 subred pública y 1 subred privada. Se empleará el bloque CIDR IPv4 10.2.0.0/16 para crear subredes 10.2.0.0/18.

##2.1 **Paso 1:** Crear la VPC

La configuración a seleccionar es la siguiente:
- Resources to create: VPC only
- IPv4 CIDR manual input:
- IPv6 CIDR block: Amazon-provided IPv6 CIDR block
- Tenancy: Default (sin HW dedicado, más barato)

##2.2 **Paso 2:** Crear las subredes

Las subredes públicas y privadas se crean igual. La diferencia estará más adelante en la tabla de enrutamiento al conectar o no con la puerta de enlace.

- Seleccione la VPC
- Identificar y planear el direccionamiento IPv4 e IPv6
- Asigne un nombre que indique si será pública o privada
- Seleccione AZ
- Seleccione IPv4 e IPv6 subnet CIDR block

##2.3 **Paso 3:** Crear el GW (Internet gateways)

El GW (*Gateway*) o puerta de enlace:
- Crear el GW
- Editarlo para conectarlo con la VPC:
 * En Actions selecionar Atach to VPC
 * Seleccionar la VPC

##2.4 **Paso 4:** Crear las Tablas de Enrutamiento

Es suficiente una tabla para las subredes públicas con el mismo GW y otra para las privadas.
- Seleccione la VPC y cree la tabla de enrutamiento
- Edite la tabla de enrutamiento para asociarla a la subred:
  - Seleccione la pestaña Subnet Associations
  - En la sección Subnet without explicit associations editar para selecionar la subred



##2.5 **Paso 5:** Ajustar las rutas en las Tablas de Enrutamiento

La tabla de enrutamiento para la subred pública debe ajustarse para que quede de la siguiente forma (el siguiente es un ejemplo, los datos deben ajustarse dependiendo de las direcciones IP):

|Destino|Destino|
|--|--|
|::/0|igw-0d7ec8f0c00a668ba|
|2600:1f10:4d0a:5700::/56|local|
|0.0.0.0/0|igw-0d7ec8f0c00a668ba|
|10.2.0.0/16|local|






#3.Enlace SSH entre cliente y servidor

En esta sección se creará el usuario *tele* en el Servidor y se habilitará para realizar una conexión remota ssh desde el Cliente.

Necesitará abrir dos consolas. Puede ser util cambiar el prompt para que refleje el nombre Cliente o Servidor a la vez que se visualice la IP. Por ejemplo, si el prompt actual es:
```
ip-172-32-42-156
```
Debe cambiarse el nombre del host en el archivo:
```
/etc/hostname
```
colocando para el servidor el nombre:
```
Server-172-32-42-156
```
Para que el cambio se refleje en el prompt, deben ejecutarse los comandos:
```
sudo hostname Server-172-32-42-156
exec bash
```

##3.1 Revisión e instalación

Verificar cliente:
```
ssh –V
```

Verificar servidor:
```
sudo systemctl status ssh
```

##3.2 En el servidor crear usuario

Crear el usuario *tele* con contraseña *tele*.

##3.3 En el servidor ajustar la configuración

En el archivo de configuración del servidor:

```
/etc/ssh/sshd_config
```

Quitarle el comentario y asignar el siguiente valor a las opciones:
- Port 22
- PasswordAuthentication yes
- PubkeyAuthentication yes
- KbdInteractiveAuthentication yes

La opción *KbdInteractiveAuthentication* debe estar en *yes* para permitir interacción remota desde teclado.
Para la autenticación en el servidor ssh hay dos opciones:
- Si *PubkeyAuthentication* está en *yes* la autenticación será por clave pública.
- Si *PubkeyAuthentication* está en *no* y *PasswordAuthentication* en *yes* la autenticación será por password.

##3.4 En el cliente generar y verificar el par de llaves

Para generar el par de llaves se sugiere:
```
ssh-keygen -t rsa -b 4096
```
Las llaves se crean en las siguientes ubicaciones:
- ~/.ssh/id_rsa	: Clave privada, no compartir.
- ~/.ssh/id_rsa.pub	: Clave pública, puede compartirse
Observe las llaves.

##3.5 Desde el cliente copiar la clave pública al servidor

El comando para enviar desde el cliente la clave pública al servidor es el siguiente:
```
 ssh-copy-id username@IP_servidor
```

Si el comando anterior no funciona, la clave debe copiarse manualmente. Siga los siguientes pasos:

- Paso 1: En el Servidor, ingrese a la sesión del usuario tele.
- Paso 2: En el directorio *home* crear el directorio *.ssh* con los siguientes privilegios:
```
 drwx------      .ssh
```
- Paso 3: En el directorio *.ssh* crear el archivo *authorized_keys* con los siguientes privilegios:
```
 -rw-------      authorized_keys
```
- Paso 4: Copie manualmente la llave pública creada en el Cliente, al archivo *authorized_keys* en el servidor.


#4.UFW
La configuración del servicio UFW (*Uncomplicated FireWall*) es la siguiente:

##4.1. Configurar las políticas por defecto
```
sudo ufw default deny incoming​
sudo ufw default allow outgoing​
```

##4.2. Habilitar servicios

El mapeo de servicios a números de puertos se encuentra en el archivo:
```
/etc/services
```
Para agregar un servicio puede hacerse por el nombre de servicio o por número de puerto:
```
sudo ufw allow servicio​
sudo ufw allow numero_puerto​
```

##4.3. Controlar UFW​
Para controlar el servicio UFW se emplean los siguientes comandos:
```
sudo ufw enable​
sudo ufw status verbose​
sudo ufw disable​
sudo ufw status numbered
```

#5.DHCP

El archivo de configuración del servicio:
```
/etc/dhcp/dhcpd.conf
```
Un ejemplo de configuración es la siguiente:
```
default-lease-time 600;
max-lease-time 7200;
subnet 192.168.1.0 netmask 255.255.255.0 {
  range 192.168.1.150 192.168.1.200;
  option routers 192.168.1.254;
  option domain-name-servers 192.168.1.1;
  option domain-name "mydomain.example";
```


#6.DNS

Se realizará un ejemplo de configuración con la siguiente información:
- Nombre de dominio: telematica.com
- Dirección de red: 10.100.1.0/24

Con esta información, los datos para el servicio son:
-	zona directa: **telematica.com**
- zona inversa: **1.100.10.in-addr.arpa**
Se seleccionan los nombres:
- Nombre de archivo directo: **a_directo**
- Nombre de archivo inverso: **a_inverso**

**Paso 1:** Instalar bind9 en el servidor:

**Paso 2:** Crear y verificar las zonas:
1. cree una copia del siguiente archivo:
```
  /etc/bind/named.conf.local
```

2. Edite el archivo:
```
  zone "telematica.com" {
    type master;
    file "a_directo"
  ; };

  zone "1.100.10.in-addr.arpa" {
    type master;
    file "a_inverso"
  ; };
```
3. Verifique la edición del archivo
```
	named-checkconf
```

**Paso 3:** Crear y verificar el archivo del mapa directo:
1. Cree el archivo del mapa directo:
```
  /var/cache/bind/a_directo
```
2. Edite
```
$ORIGIN telematica.com.
$TTL 86400 ; 1 dia
@ IN SOA servidor postmaster (
120219 ; serie
6H ; refrescamiento (6 horas)
1H ; reintentos (1 hora)
2W ; expira (2 semanas)
3H ; mínimo cache (3 horas)
)
      IN NS server
server IN A 10.100.1.10
client IN A 10.100.1.20
```
3. Verifique la edición del archivo
La sintaxis es:
```  
  named-checkzone zona_directa ruta/a_directo
```

**Paso 4:** Crear y verificar el archivo del mapa inverso:
1. Cree el archivo del mapa directo:
```
  /var/cache/bind/a_inverso
```
2. Edite
```
$ORIGIN 1.100.10.in-addr.arpa.
$TTL 86400 ; 1 dia
@ IN SOA servidor postmaster (
120219 ; serie
6H ; refresco (6 horas)
1H ; reintentos (1 hora)
2W ; expira (2 semanas)
3H ; mínimo (3 horas)
)
      IN NS server.telematica.com.
10 IN PTR server.telematica.com.
20 IN PTR client.telematica.com.
```
3. Verifique la edición del archivo
```  
  named-checzone zona_inversa ruta/a_inverso
```

**Paso 5:** Permitir la resolución externa

Para resolver nombres de dominios externos debe editarse el archivo:
```
  /etc/bind/named.conf.options
```
Verificar:
```
options {
    directory "/var/cache/bind";

    recursion yes;
    allow-recursion { any; };   #resolución externa

    forwarders {
        8.8.8.8;   # DNS de Google
        8.8.4.4;   # DNS de Google
    };
```

**Paso 6:** Reiniciar el servidor:
```
 	sudo systemctl restart bind9
```

Para observar el estado del servidor:
```
 	sudo systemctl status bind9
```
